# Colab 환경 실행

[![Open in Coab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jsk1107/dacon_k_fashion/blob/master/Colab_NoteBook.ipynb)

## Colab Session 연장

### 아래 코드를 F12를 누른 후 Console에 입력할것 

function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);

In [ ]:
!git clone https://github.com/jsk1107/dacon_k_fashion